# LightGBM Booking Probability Prediction with Optuna Tuning
This notebook demonstrates how to train a LightGBM model that predicts the probability a hotel will be booked. We use Optuna to tune hyperparameters with cross‑validation, evaluate the tuned model, and optionally explain it with SHAP.

## 1. Imports

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import optuna
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score, log_loss
import shap
import warnings
warnings.filterwarnings('ignore')


/opt/homebrew/anaconda3/envs/fenics-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Configuration
Adjust the paths and parameters below to match your environment and dataset.

In [2]:
# === File paths ===
TRAIN_CSV = '/Users/s.broos/Documents/DMT_data/training_set_VU_DM.csv'            # TODO: replace with your training file
OUT_FEAT_CSV = '/Users/s.broos/Documents/DMT_data/features_with_cf_score.csv'  # TODO: replace with your engineered‑features file

# === Column names ===
LABEL_CLICK = 'click_bool'
LABEL_BOOK = 'booking_bool'
LEAKS = ['position', 'gross_bookings_usd']     # columns that leak target information
REMOVE_COLS = []                               # extra columns you want dropped

# === Sampling ===
SAMPLE_FRAC = 1.0   # set <1.0 for quick tests
RANDOM_STATE = 42


## 3. Helper functions

In [3]:
def engineer_light(df: pd.DataFrame) -> pd.DataFrame:
    """Simple numeric imputation with medians."""
    df = df.copy()
    num_cols = df.select_dtypes('number').columns
    df[num_cols] = df[num_cols].fillna(df[num_cols].median())
    return df


## 4. Load & prepare data

In [4]:
# 4.1 Load raw data
df_full = pd.read_csv(TRAIN_CSV)
df_full = engineer_light(df_full)

# 4.2 Labels
y = df_full[LABEL_BOOK]

# 4.3 Base feature matrix
drop_cols = [LABEL_CLICK, LABEL_BOOK] + LEAKS + ['date_time']
X_base = df_full.drop(columns=drop_cols)
X_base = X_base.drop(columns=[c for c in REMOVE_COLS if c in X_base.columns])

# 4.4 Merge candidate engineered features
cand = pd.read_csv(OUT_FEAT_CSV)
X_full = X_base.merge(cand, on=['srch_id', 'prop_id'], how='left')

# 4.5 (Optional) sample for speed
X_full = X_full.sample(frac=SAMPLE_FRAC, random_state=RANDOM_STATE)
y = y.loc[X_full.index].reset_index(drop=True)
X_full = X_full.reset_index(drop=True)

print(f'Full feature matrix shape: {X_full.shape}')


Full feature matrix shape: (4958347, 66)


### 4.6 Train / test split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X_full, y,
    test_size=0.3,
    stratify=y,
    random_state=RANDOM_STATE
)
print(f'Train size: {X_train.shape},  Test size: {X_test.shape}')


Train size: (3470842, 66),  Test size: (1487505, 66)


## 5. Hyperparameter optimisation with Optuna

In [14]:
# ------------------------------------------------------------------
# Fast LightGBM + Optuna tuning block
# ------------------------------------------------------------------
import lightgbm as lgb
import optuna
from optuna.integration import LightGBMPruningCallback
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# ---------- fast-tune hyper-parameters ----------
SAMPLE_FRAC   = 0.60      # more data
NUM_BOOST_CAP = 1200      # more trees
EARLY_STOP    = 40        # patience
N_TRIALS      = 30        # more Optuna shots

# ---------- one-time sampling for speed ----------
X_tune, _, y_tune, _ = train_test_split(
    X_train, y_train,
    test_size=1 - SAMPLE_FRAC,
    stratify=y_train,
    random_state=RANDOM_STATE
)

# ---------- Optuna objective ----------
def objective(trial):
    # --- 80 / 20 validation split ---
    X_tr, X_val, y_tr, y_val = train_test_split(
        X_tune, y_tune,
        test_size=0.20,
        stratify=y_tune,
        random_state=RANDOM_STATE
    )
    dtr  = lgb.Dataset(X_tr,  label=y_tr)
    dval = lgb.Dataset(X_val, label=y_val, reference=dtr)

    # --- compact search space ---
    params = {
        "objective": "binary",
        "metric": "auc",
        "verbosity": -1,
        "seed": RANDOM_STATE,
        "learning_rate": trial.suggest_float("learning_rate", 0.02, 0.2, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 31, 255, log=True),
        "min_child_samples": trial.suggest_int("min_child_samples", 20, 100),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 1.0),
        "bagging_freq": 1,
        # "device_type": "gpu",   # ← uncomment if you have CUDA
    }

    booster = lgb.train(
        params,
        dtr,
        num_boost_round=NUM_BOOST_CAP,
        valid_sets=[dval],
        callbacks=[
            lgb.early_stopping(EARLY_STOP),
            LightGBMPruningCallback(trial, "auc"),
            lgb.log_evaluation(period=0),   # silent
        ],
    )

    preds = booster.predict(X_val, num_iteration=booster.best_iteration)
    return roc_auc_score(y_val, preds)

# ---------- run the study ----------
study = optuna.create_study(direction="maximize")
study.optimize(
    objective,
    n_trials=N_TRIALS,
    n_jobs=-1,                # use all available CPU cores
    show_progress_bar=True
)

print("Best AUC (CV):", study.best_value)
print("Best params :", study.best_params)


[I 2025-05-14 17:40:58,313] A new study created in memory with name: no-name-b81805c3-2bbe-45b7-a37b-6f13fdc3989d
  0%|          | 0/30 [00:00<?, ?it/s]

Training until validation scores don't improve for 40 rounds
Training until validation scores don't improve for 40 rounds
Training until validation scores don't improve for 40 rounds
Training until validation scores don't improve for 40 rounds
Training until validation scores don't improve for 40 rounds
Training until validation scores don't improve for 40 rounds
Training until validation scores don't improve for 40 rounds
Training until validation scores don't improve for 40 rounds
Early stopping, best iteration is:
[27]	valid_0's auc: 0.913346


Best trial: 1. Best value: 0.913346:   3%|▎         | 1/30 [01:30<43:51, 90.74s/it]

[I 2025-05-14 17:42:29,046] Trial 1 finished with value: 0.913346239236184 and parameters: {'learning_rate': 0.16866861590324983, 'num_leaves': 147, 'min_child_samples': 28, 'feature_fraction': 0.7426796664344582, 'bagging_fraction': 0.6322446513624453}. Best is trial 1 with value: 0.913346239236184.
Early stopping, best iteration is:
[78]	valid_0's auc: 0.917013


Best trial: 5. Best value: 0.917013:   7%|▋         | 2/30 [01:45<21:21, 45.76s/it]

...

[I 2025-05-14 17:48:13,314] Trial 29 pruned. Trial was pruned at iteration 1.
Early stopping, best iteration is:
[714]	valid_0's auc: 0.91872


Best trial: 2. Best value: 0.918726:  97%|█████████▋| 29/30 [07:26<00:05,  5.48s/it]

[I 2025-05-14 17:48:24,642] Trial 7 finished with value: 0.9187197118735517 and parameters: {'learning_rate': 0.0265228313275646, 'num_leaves': 50, 'min_child_samples': 63, 'feature_fraction': 0.8695287429670948, 'bagging_fraction': 0.667308395865633}. Best is trial 2 with value: 0.9187261423038111.
Early stopping, best iteration is:
[785]	valid_0's auc: 0.919291


Best trial: 4. Best value: 0.919291: 100%|██████████| 30/30 [07:57<00:00, 15.91s/it]

[I 2025-05-14 17:48:55,472] Trial 4 finished with value: 0.9192906198175425 and parameters: {'learning_rate': 0.02340256277784342, 'num_leaves': 72, 'min_child_samples': 69, 'feature_fraction': 0.6719019183257495, 'bagging_fraction': 0.8929756582704587}. Best is trial 4 with value: 0.9192906198175425.
Best AUC (CV): 0.9192906198175425
Best params : {'learning_rate': 0.02340256277784342, 'num_leaves': 72, 'min_child_samples': 69, 'feature_fraction': 0.6719019183257495, 'bagging_fraction': 0.8929756582704587}


Best trial: 4. Best value: 0.919291: 100%|██████████| 30/30 [07:57<00:00, 15.91s/it]
[I 2025-05-14 17:48:55,472] Trial 4 finished with value: 0.9192906198175425 and parameters: {'learning_rate': 0.02340256277784342, 'num_leaves': 72, 'min_child_samples': 69, 'feature_fraction': 0.6719019183257495, 'bagging_fraction': 0.8929756582704587}. Best is trial 4 with value: 0.9192906198175425.
Best AUC (CV): 0.9192906198175425
Best params : {'learning_rate': 0.02340256277784342, 'num_leaves': 72, 'min_child_samples': 69, 'feature_fraction': 0.6719019183257495, 'bagging_fraction': 0.8929756582704587}


## 6. Train final model with tuned parameters

In [18]:
# ============ FINAL TRAINING (all data) ============

import lightgbm as lgb
import joblib, os

# 1. Merge every row you have
X_full = pd.concat([X_train, X_test]).reset_index(drop=True)
y_full = pd.concat([y_train, y_test]).reset_index(drop=True)
dall   = lgb.Dataset(X_full, label=y_full)

# 2. Params from Optuna
best_params = study.best_params.copy()
best_params.update({
    "objective": "binary",
    "metric": ["auc", "binary_logloss"],
    "verbosity": -1,
    "seed": RANDOM_STATE,
})

cv_results = lgb.cv(
    best_params,
    dall,
    num_boost_round=4000,          # hard cap
    nfold=3,                       # 3 folds instead of 5
    stratified=True,
    shuffle=True,
    callbacks=[
        lgb.early_stopping(100),   # patience halved
        lgb.log_evaluation(period=200),
    ],
)
best_iter = len(cv_results["auc-mean"])
print(f"\nBest_iteration from CV: {best_iter}")

# 4. Fit one model on 100 % of the data
gbm_final = lgb.train(
    best_params,
    dall,
    num_boost_round=best_iter,
    valid_sets=[dall],
    verbose_eval=False
)

# 5. Save for production use
MODEL_PATH = "lightgbm_full_final.pkl"
joblib.dump(gbm_final, MODEL_PATH)
print(f"✅  Model saved to {os.path.abspath(MODEL_PATH)}")


KeyboardInterrupt: 

In [ ]:
# ============ EVALUATION (OOF metrics) ============

import numpy as np
from sklearn.metrics import (
    roc_auc_score, average_precision_score, log_loss,
    brier_score_loss, accuracy_score, precision_score,
    recall_score, f1_score, confusion_matrix
)

# 1. Collect OOF predictions from the cvbooster
oof_pred = np.zeros(len(X_full))
for booster, (_, val_idx) in zip(cv_results["cvbooster"].boosters, cv_results["cvbooster"].folds):
    oof_pred[val_idx] = booster.predict(
        X_full.iloc[val_idx],
        num_iteration=booster.best_iteration
    )

# 2. Threshold → class labels (0.50 default)
y_pred = (oof_pred >= 0.50).astype(int)

# 3. Metrics table
print("──────────  CV Out-of-Fold Metrics  ──────────")
print(f"AUC-ROC           : {roc_auc_score(y_full, oof_pred):.4f}")
print(f"PR-AUC            : {average_precision_score(y_full, oof_pred):.4f}")
print(f"Log-loss          : {log_loss(y_full, oof_pred):.4f}")
print(f"Brier score       : {brier_score_loss(y_full, oof_pred):.4f}")
print("---------------------------------------------")
print(f"Accuracy (0.5)    : {accuracy_score(y_full, y_pred):.4f}")
print(f"Precision (0.5)   : {precision_score(y_full, y_pred, zero_division=0):.4f}")
print(f"Recall    (0.5)   : {recall_score(y_full, y_pred, zero_division=0):.4f}")
print(f"F1-score  (0.5)   : {f1_score(y_full, y_pred, zero_division=0):.4f}")
tn, fp, fn, tp = confusion_matrix(y_full, y_pred).ravel()
print(f"Confusion matrix  :  TN={tn}  FP={fp}  FN={fn}  TP={tp}")
print("──────────────────────────────────────────────")


## 7. Interpretability with SHAP
Uncomment the summary‑plot line if you wish to visualise SHAP values inside the notebook.

In [ ]:
explainer = shap.TreeExplainer(gbm_final)
shap_values = explainer.shap_values(X_test.fillna(0))

# shap.summary_plot(shap_values, X_test, max_display=20)


---
*End of notebook*